In [1]:
from xpbd_softbody import XPBDSoftbody
import pyvista as pv
import config as cfg
import matplotlib.pyplot as plt
import torch
from xpbd_softbody_layer import XPBDStep
import numpy as np
from sklearn.neighbors import KDTree
from xpbd_softbody_layer import get_energy_boundary
from scipy.interpolate import interp1d
from cubic_bezier import *
from tqdm import trange
from torchviz import make_dot
import os
from tqdm import trange

In [2]:
path='../assets'
softbody = XPBDSoftbody()
mesh = softbody.add_thinshell(pv.read(os.path.join(path, 'grape_skin.ply')), n_surf=cfg.n_surf)
mesh = softbody.add_thinshell(pv.read(os.path.join(path, 'grape_skin.ply')), n_surf=cfg.n_surf)
# mesh = softbody.add_thinshell(pv.Plane(), n_surf=cfg.n_surf)
softbody.init_states()
softbody.init_dist_constraints()

softbody.add_multi_boundary_constrain(1, 0, 0.0006, range(600))
softbody.fix_indice(0, range(600))

detect 2 object


In [4]:
softbody.V

tensor([[-0.0002,  0.0002,  0.0014],
        [-0.0004, -0.0004,  0.0014],
        [-0.0006, -0.0011,  0.0014],
        ...,
        [ 0.0016,  0.0122, -0.0023],
        [ 0.0011,  0.0126, -0.0025],
        [ 0.0005,  0.0130, -0.0026]])

In [3]:
softbody.C_boundary_list[0].shape

torch.Size([690, 2])

In [4]:
V_origin = softbody.V.clone()
V_velocity_origin = softbody.V_velocity.clone()

In [5]:
# stiffness setting
cloth_dist_stiffness = 1
V_boundary_stiffness = 0.1
V_dist_stiffness = torch.ones_like(softbody.V_mass).to(cfg.device)
V_boundary_stiffness = torch.ones((softbody.C_boundary_list[0].shape[0], 1)).to(cfg.device) * V_boundary_stiffness

In [6]:
V_boundary_stiffness.shape

torch.Size([690, 1])

In [7]:
u = torch.linspace(0, 1, 3).to(cfg.device)
us = torch.linspace(0, 1, 50).to(cfg.device)
start_point = np.array([[0.000224, 0.010794, -0.001233]])
start_point = torch.from_numpy(start_point).to(cfg.device)
# spline_control = np.array([ [0.000224, 0.010794, 0.003],
#                             [0.000224, 0.010794, 0.006],
#                             [0.000224, 0.010794, 0.009]])
spline_control = np.array([ [0.000224, 0.009794, 0.001],
                            [0.000224, 0.008794, 0.0020],
                            [0.000224, 0.007794, 0.0035]])
spline_control = torch.from_numpy(spline_control).to(cfg.device)

In [8]:
control_point = 31 + softbody.offset_list[1]
softbody.fix_point(1, 31)

In [9]:
V_boundary_stiffness.shape

torch.Size([690, 1])

In [10]:
pv.set_plot_theme('document')
pl = pv.Plotter()

# skin being peeled
pl.add_mesh(mesh, color='#9f5547ff', show_edges=True, edge_color='#b37164ff',  lighting=False,style='surface')
# ellipsoid grape
grape = pv.read('../assets/grape.ply')
grape.points = grape.points - np.array([0, 0, 1e-4])
pl.add_mesh(grape, color='#9f5547ff', show_edges=False, lighting=False,style='surface')

# grape meat that is green
grape_meat= pv.read('../assets/grape_skin.ply')
grape_meat.points = grape_meat.points - np.array([0, 0, 2e-4])
pl.add_mesh(grape_meat, color='#c0ab5eff', show_edges=False, lighting=False,style='surface')

Actor (0x7fa7f5e616c0)
  Center:                     (-9.851530194282532e-06, 0.0014494163915514946, -0.0009795548299327494)
  Pickable:                   True
  Position:                   (0.0, 0.0, 0.0)
  Scale:                      (1.0, 1.0, 1.0)
  Visible:                    True
  X Bounds                    -6.636E-03, 6.616E-03
  Y Bounds                    -1.040E-02, 1.330E-02
  Z Bounds                    -3.189E-03, 1.230E-03
  User matrix:                Set
  Has mapper:                 True

Property (0x7fa7f5e61ae0)
  Ambient:                     0.0
  Ambient color:               Color(hex='#c0ab5eff', opacity=255)
  Anisotropy:                  0.0
  Color:                       Color(hex='#c0ab5eff', opacity=255)
  Culling:                     "none"
  Diffuse:                     1.0
  Diffuse color:               Color(hex='#c0ab5eff', opacity=255)
  Edge color:                  Color(name='black', hex='#000000ff', opacity=255)
  Edge opacity:                1.0
 

In [11]:
# setup opt para
spline_control.requires_grad_(True)

tensor([[0.0002, 0.0098, 0.0010],
        [0.0002, 0.0088, 0.0020],
        [0.0002, 0.0078, 0.0035]], dtype=torch.float64, requires_grad=True)

In [12]:
optimizer = torch.optim.Adam([spline_control], lr = 0.001)

In [13]:
## point position target
# target = torch.tensor([0.01, 0.01, 0.01])
# target_point = 25 + softbody.offset_list[1]

In [14]:
## stiffness target
target = torch.ones_like(V_boundary_stiffness) * 1e-5

In [15]:
def loss_fn(target, pred):
    return torch.norm(target - pred)

In [16]:
# pl.open_gif('grape.gif')
for t in range(10):
    x_con = torch.cat((start_point[:, 0], spline_control[:, 0]))
    y_con = torch.cat((start_point[:, 1], spline_control[:, 1]))
    z_con = torch.cat((start_point[:, 2], spline_control[:, 2]))

    spline = cubic_bezier_arc_3D(x_con, y_con, z_con, 0.00025)
    print(spline.shape)
    spline_x = spline[0]
    spline_y = spline[1]
    spline_z = spline[2]

    spline_trajectory = torch.transpose(torch.vstack((spline_x, spline_y, spline_z)), 0, 1)
    # reset for each test
    softbody.V = V_origin.clone()
    softbody.V_velocity = V_velocity_origin.clone()

    cloth_dist_stiffness = 1
    V_boundary_stiffness = 0.1
    V_dist_stiffness = torch.ones_like(softbody.V_mass).to(cfg.device)
    V_boundary_stiffness = torch.ones((softbody.C_boundary_list[0].shape[0], 1)).to(cfg.device) * V_boundary_stiffness
    energy_total = torch.zeros(1).to(cfg.device)
    for i in trange(spline_trajectory.shape[0]):
        softbody.V[control_point] = spline_trajectory[i]
        step_ref = XPBDStep(softbody,
                        V_dist_stiffness=V_dist_stiffness, 
                        V_shape_stiffness=None,
                        V_boundary_stiffness=V_boundary_stiffness, 
                        dt=cfg.dt,
                        substep=cfg.substep,
                        iteration=cfg.iteration,
                        quasi_static=cfg.quasi_static,
                        plane_height=cfg.ground_plane_height, 
                        use_shape_matching=cfg.use_shape_matching,
                        use_spring_boundary=cfg.use_spring_boundary,
                        use_dist=True) #cfg.use_spring_boundary
        V_ref, V_velocity_ref = step_ref.forward(softbody.V, softbody.V_velocity)
        softbody.V = V_ref.clone()
        softbody.V_velocity = V_velocity_ref.clone()
        ref_V_boundary_stiffness = V_boundary_stiffness.clone()
        energy = get_energy_boundary(softbody, softbody.V, ref_V_boundary_stiffness)
        # set boundary energy that larger than 1e-8 as broken, 1e-5 is a eps to avoid divide 0
        # 1e-8 is the threshold, if energy is larger than 1e-8, the first term will be 0 and the last term will be 1e-5 * 1.
        V_boundary_stiffness = ref_V_boundary_stiffness * torch.sigmoid(1e10 * (1e-8 - energy)) + 1e-5*torch.sigmoid(1e10 * (energy - 1e-8))
        mesh.points = softbody.V[-600:].detach().cpu().numpy()
    
    #optimize
    loss = loss_fn(target, V_boundary_stiffness)
    loss.backward()
    print(spline_control.grad)
    optimizer.step()
    optimizer.zero_grad()
    # pl.write_frame()
# pl.close()


In [17]:
step_ref = XPBDStep(softbody,
                    V_dist_stiffness=V_dist_stiffness, 
                    V_shape_stiffness=None,
                    V_boundary_stiffness=V_boundary_stiffness, 
                    dt=cfg.dt,
                    substep=cfg.substep,
                    iteration=cfg.iteration,
                    quasi_static=cfg.quasi_static,
                    plane_height=cfg.ground_plane_height, 
                    use_shape_matching=cfg.use_shape_matching,
                    use_spring_boundary=cfg.use_spring_boundary,
                    use_dist=True) #cfg.use_spring_boundary

In [18]:
softbody.C_boundary_V_0

[tensor([ 600,  601,  602,  603,  604,  605,  606,  607,  608,  609,  610,  611,
          612,  613,  614,  615,  616,  617,  618,  619,  620,  621,  622,  623,
          624,  625,  626,  627,  628,  629,  630,  631,  632,  633,  634,  635,
          636,  637,  638,  639,  640,  641,  642,  643,  644,  645,  646,  647,
          648,  649,  650,  651,  652,  653,  654,  655,  656,  657,  658,  659,
          660,  661,  662,  663,  664,  665,  666,  667,  668,  669,  670,  671,
          672,  673,  674,  675,  676,  677,  678,  679,  680,  681,  682,  683,
          684,  685,  686,  687,  688,  689,  690,  691,  692,  693,  694,  695,
          696,  697,  698,  699,  700,  701,  702,  703,  704,  705,  706,  707,
          708,  709,  710,  711,  712,  713,  714,  715,  716,  717,  718,  719,
          720,  721,  722,  723,  724,  725,  726,  727,  728,  729,  730,  731,
          732,  733,  734,  735,  736,  737,  738,  739,  740,  741,  742,  743,
          744,  745,  746,  

In [19]:
step_ref.project_list

[project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_dist(),
 project_C_spring_boundary()]

In [20]:
pl.open_gif('grape.gif')
x_con = torch.cat((start_point[:, 0], spline_control[:, 0]))
y_con = torch.cat((start_point[:, 1], spline_control[:, 1]))
z_con = torch.cat((start_point[:, 2], spline_control[:, 2]))

# spline_x = cubic_bezier(x_con, us)
# spline_y = cubic_bezier(y_con, us)
# spline_z = cubic_bezier(z_con, us)

# spline_x = interp(u, x_con, us)
# spline_y = interp(u, y_con, us)
# spline_z = interp(u, z_con, us)

spline = cubic_bezier_arc_3D(x_con, y_con, z_con, 0.00025)
print(spline.shape)
spline_x = spline[0]
spline_y = spline[1]
spline_z = spline[2]

spline_trajectory = torch.transpose(torch.vstack((spline_x, spline_y, spline_z)), 0, 1)
# reset for each test
softbody.V = V_origin.clone()
softbody.V_velocity = V_velocity_origin.clone()

cloth_dist_stiffness = 1
V_boundary_stiffness = 0.1
V_dist_stiffness = torch.ones_like(softbody.V_mass).to(cfg.device)
V_boundary_stiffness = torch.ones((softbody.C_boundary_list[0].shape[0], 1)).to(cfg.device) * V_boundary_stiffness
energy_total = torch.zeros(1).to(cfg.device)
for i in trange(spline_trajectory.shape[0]):
    softbody.V[control_point] = spline_trajectory[i]
    step_ref = XPBDStep(softbody,
                    V_dist_stiffness=V_dist_stiffness, 
                    V_shape_stiffness=None,
                    V_boundary_stiffness=V_boundary_stiffness, 
                    dt=cfg.dt,
                    substep=cfg.substep,
                    iteration=cfg.iteration,
                    quasi_static=cfg.quasi_static,
                    plane_height=cfg.ground_plane_height, 
                    use_shape_matching=cfg.use_shape_matching,
                    use_spring_boundary=cfg.use_spring_boundary,
                    use_dist=True) #cfg.use_spring_boundary
    V_ref, V_velocity_ref = step_ref.forward(softbody.V, softbody.V_velocity)
    softbody.V = V_ref.clone()
    softbody.V_velocity = V_velocity_ref.clone()
    ref_V_boundary_stiffness = V_boundary_stiffness.clone()
    energy = get_energy_boundary(softbody, softbody.V, ref_V_boundary_stiffness)
    # set boundary energy that larger than 1e-8 as broken, 1e-5 is a eps to avoid divide 0
    # 1e-8 is the threshold, if energy is larger than 1e-8, the first term will be 0 and the last term will be 1e-5 * 1.
    V_boundary_stiffness = ref_V_boundary_stiffness * torch.sigmoid(1e10 * (1e-8 - energy)) + 1e-5*torch.sigmoid(1e10 * (energy - 1e-8))
    mesh.points = softbody.V[-600:].detach().cpu().numpy()
    
    pl.write_frame()
pl.close()


torch.Size([3, 23])


  0%|          | 0/23 [00:00<?, ?it/s]

  4%|▍         | 1/23 [00:01<00:32,  1.46s/it]

  9%|▊         | 2/23 [00:02<00:28,  1.34s/it]

 13%|█▎        | 3/23 [00:03<00:26,  1.30s/it]

...

 83%|████████▎ | 19/23 [00:23<00:04,  1.23s/it]

 87%|████████▋ | 20/23 [00:24<00:03,  1.21s/it]

 91%|█████████▏| 21/23 [00:25<00:02,  1.19s/it]

 96%|█████████▌| 22/23 [00:26<00:01,  1.17s/it]

100%|██████████| 23/23 [00:27<00:00,  1.21s/it]
